# Music Genre Classification with Deep Learning

This tutorial shows how different Convolutional Neural Network architectures are used for the taks of music genre classification.

The data set used is the [GTZAN](http://marsyasweb.appspot.com/download/data_sets/) genre data set compiled by George Tzanetakis. It consists of 1000 tracks (30 second excerpts) from 10 genres, each with 100 examples.

The original tracks are 22050Hz Mono 16-bit audio files in .au format.

For a more compact download we provide a version in .mp3 format, also 22050 Hz.

This tutorial contains:
* Loading and preprocessing of audio files
* Loading class files from CSV and using Label Encoder
* using One-Hot Encoder to prepare class data for Deep Learning
* Generating Mel spectrograms from the audio
* Standardization of data
* Train/Test set split
* Stratified splits
* Convolutional Neural Networks: single, stacked, parallel
* ReLU Activation
* Dropout
* Batch Normalization

(entensions for later:)
* Majority Vote / Max Probability 
* Recurrent Neural Networks
* Cross-validation

You can execute the following code blocks by pressing SHIFT+Enter consecutively.

In [1]:
import os

# choosing between CPU and GPU
#device = 'cpu' # 'cpu' or 'gpu'
#os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=' + device + ',floatX=float32'

import argparse
import csv
import datetime
import glob
import math
import sys
import time
import numpy as np
import pandas as pd # Pandas for reading CSV files and easier Data handling in preparation
from os.path import join

from theano import config

import keras
from keras.models import Sequential, Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

# local
import rp_extract as rp
from audiofile_read import audiofile_read

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn import __version__ as sklearn_version

if sklearn_version.startswith('0.17'):
    from sklearn.cross_validation import train_test_split
    from sklearn.cross_validation import StratifiedShuffleSplit
else: # >= 0.18
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import StratifiedShuffleSplit

Using Theano backend.


## Set the Path to the Dataset

adjust this path to where the data set is stored on your computer:


In [2]:
# SET YOUR OWN PATH HERE
AUDIO_PATH = '../data/GTZAN_mp3'

## Load the Metadata

The tab-separated file contains pairs of filename TAB class category (i.e. genre).

In [3]:
csv_file = join(AUDIO_PATH,'filelist_GTZAN_mp3_wclasses.txt')
metadata = pd.read_csv(csv_file, index_col=0, header=None)
metadata.head(10)

,1
0,
./rock/rock.00053.mp3,rock
./rock/rock.00051.mp3,rock
./rock/rock.00076.mp3,rock
./rock/rock.00084.mp3,rock
./rock/rock.00052.mp3,rock
./rock/rock.00057.mp3,rock
./rock/rock.00028.mp3,rock
./rock/rock.00035.mp3,rock
./rock/rock.00095.mp3,rock


In [4]:
# create list of filenames with associated classes
filelist = metadata.index.tolist()
classes = metadata[1].values.tolist()

## Encode Labels to Numbers

String labels need to be encoded as numbers. We use the LabelEncoder from the scikit-learn package.

In [5]:
classes[0:5]

['rock', 'rock', 'rock', 'rock', 'rock']

In [6]:
classes[99:105]

['rock', 'hiphop', 'hiphop', 'hiphop', 'hiphop', 'hiphop']

In [7]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
labelencoder.fit(classes)

# we keep (and print) the number of classis
n_classes = len(labelencoder.classes_)
print n_classes, "classes:", ", ".join(list(labelencoder.classes_))

classes_num = labelencoder.transform(classes)

10 classes: blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock


We check how the classes look like now numerically:

In [8]:
classes_num[0:5]

array([9, 9, 9, 9, 9])

In [9]:
classes_num[99:105]

array([9, 4, 4, 4, 4, 4])

Note: In order to correctly re-transform any predicted numbers into strings, we keep the labelencoder for later.

In [10]:
from sklearn.preprocessing import OneHotEncoder

# make a row vector a column vector, as needed by OneHotEncoder, using reshape(-1,1) 
classes_num_col = classes_num.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
classes_num_1hot = encoder.fit_transform(classes_num_col)
classes_num_1hot

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0., 

In [11]:
classes_num_1hot.shape

(1000, 10)

## Load the Audio Files

In [23]:

list_spectrograms = [] # spectrograms are put into a list first

# desired output parameters
n_mel_bands = 40   # y axis
frames = 80        # x axis

# some FFT parameters
fft_window_size=512
fft_overlap = 0.5
hop_size = int(fft_window_size*(1-fft_overlap))
segment_size = fft_window_size + (frames-1) * hop_size # segment size for desired # frames

for filename in filelist:
    print ".", 
    filepath = os.path.join(AUDIO_PATH, filename)
    samplerate, samplewidth, wavedata = audiofile_read(filepath,verbose=False)
    sample_length = wavedata.shape[0]

    # make Mono (in case of multiple channels / stereo)
    if wavedata.ndim > 1:
        wavedata = np.mean(wavedata, 1)
      
    # GET AUDIO SEGMENT (BLOCK)
    # take only 1 audio segment of 0.94 sec
    # a) from beginning of file
    #pos = 0
    # b) from middle of file
    pos = int(sample_length / 2 - segment_size / 2)
    wav_segment = wavedata[pos:pos+segment_size]
    
    # AUDIO PRE-PROCESSING

    # 1) FFT spectrogram 
    spectrogram = rp.calc_spectrogram(wav_segment,fft_window_size,fft_overlap)

    # 2) Transform to perceptual Mel scale (uses librosa.filters.mel)
    spectrogram = rp.transform2mel(spectrogram,samplerate,fft_window_size,n_mel_bands)
        
    # 3) Log 10 transform
    spectrogram = np.log10(spectrogram)
    
    list_spectrograms.append(spectrogram)
        
print "\nRead", len(filelist), "audio files"

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [24]:
print "Duration of input audio file:"
print sample_length / samplerate, "seconds"

Duration of input audio file:
30 seconds


In [25]:
print "An audio segment is", round(float(segment_size) / samplerate, 2), "seconds long"

An audio segment is 0.94 seconds long


In [26]:
# how many spectrograms did we get
len(list_spectrograms)

1000

In [27]:
# whats the shape of 1 spectrogram: 40 Mel bands by 80 frames
spectrogram.shape

(40, 80)

Note: For simplicity of this tutorial, here we load only 1 single segment of ~ 1 second length from each audio file.
In a real setting, one would create training instances of as many audio segments as possible to be fed to a Neural Network.

In [28]:
spectrogram[0:5,0:5]

array([[-3.19312881, -2.46611857, -2.33069044, -2.78888361, -2.65921513],
       [-2.81906141, -2.14093101, -2.20024035, -2.72672749, -2.35348157],
       [-1.95557674, -1.92403767, -2.03611306, -2.07349593, -1.97247792],
       [-2.02953207, -2.05230601, -1.98668831, -2.02436922, -2.00982927],
       [-2.09251568, -2.77670719, -2.43870339, -2.29348992, -3.07332277]])

In [81]:
# TODO plot spectrogram

## Make 1 big array of list of spectrograms

In [24]:
# a list of many 40x80 spectrograms is made into 1 big array
# config.floatX is from Theano configration to enforce float32 precision (needed for GPU computation)
data = np.array(list_spectrograms, dtype=config.floatX)
data.shape

(1000, 40, 80)

In [50]:
# check for Inf values

# np.log10(spectrogram) will produce -inf if a spectrogram value is 0. we replace -inf by 0 here

if np.any(np.isinf(data)):
    print "Warning: Data contains inf values. Replacying by 0."
    data[np.isinf(data)] = 0

## Standardization

<b>Always normalize / standardize</b> the data before feeding it into the Neural Network!

We use <b>attribute-wise standardization</b>, i.e. each feature (i.e. 'pixel' in the spectrogram) is standardized individually, as opposed to computing a single mean and single standard deviation of all values.

(Instead of 'attribute-wise, also 'flat' standardization would also be possible,computing the mean and standard deviation across all pixels).

One possibility is 'Min-Max normalization', i.e. scaling the values between 0 and 1.

Here we use <b>Zero-mean Unit-variance standardization</b> (also known as Z-score normalization).

We use the StandardScaler from the scikit-learn package for our purpose, which performs a Zero-mean Unit-variance standardization.

In [51]:
# Scalers and normalizers work on vectors. So we have to transfor the matrix of our spectrograms into vector data
# ('vectorize' or 'reshape' them).

# vectorize
N, ydim, xdim = data.shape
data = data.reshape(N, xdim*ydim)
data.shape

ValueError: need more than 2 values to unpack

In [52]:
data[0:1].shape

(1, 3200)

In [53]:
# standardize
scaler = preprocessing.StandardScaler()
data = scaler.fit_transform(data)

Now all the values are transformed into the 0-mean 1-variance space.

In [58]:
np.mean(data, axis=0)

array([ 0.        ,  0.        ,  0.00003291, -0.00000348, -0.00000135, -0.00000497, -0.0000019 ,  0.00000086, -0.00000034, -0.000001  , ..., -0.00000007,  0.00000116, -0.00000242, -0.00000083,
       -0.00000178,  0.00000348,  0.00000149,  0.00000025, -0.00000021,  0.00000247], dtype=float32)

In [59]:
np.std(data, axis=0)

array([ 0.        ,  0.        ,  1.00000322,  0.99999994,  0.99999982,  0.99999994,  1.        ,  1.        ,  1.        ,  1.00000024, ...,  1.00000012,  1.00000024,  1.00000024,  1.00000024,
        1.        ,  0.99999988,  0.99999952,  1.        ,  0.99999964,  1.00000012], dtype=float32)

In [54]:
# scaler stores the original values to be able to transform later again
# show mean and standard deviation: two vectors with same length as data.shape[1]
scaler.mean_, scaler.scale_

(array([  0.        ,   0.        , -15.83200645, -13.3323307 , -11.67720318,  -5.16446209,  -3.19460917,  -3.1655395 ,  -3.19477248,  -3.2418499 , ...,  -8.34243298,  -8.35699749,  -8.34202576,
         -8.32057381,  -8.31046009,  -8.28858852,  -8.29681206,  -8.29219341,  -8.30333328,  -8.30508804], dtype=float32),
 array([ 1.        ,  1.        ,  1.7693063 ,  1.36666048,  1.36915123,  1.21423578,  1.23309195,  1.27106905,  1.36281705,  1.50109375, ...,  2.0394578 ,  2.01078248,  2.03702593,  2.05707383,
         2.05797529,  2.05667782,  2.06773233,  2.05173779,  2.05352616,  2.06124711], dtype=float32))

# Creating Train & Test Set 

We split the original full data set into two parts: Train Set (75%) and Test Set (25%).

Here we compare Random Split vs. Stratified Split:

In [60]:
testset_size = 0.25 # % portion of whole data set to keep for testing, i.e. 75% is used for training

# RANDOM split of data set into 2 parts
# from sklearn.model_selection import train_test_split

train_set, test_set, train_classes, test_classes = train_test_split(data, classes_num, test_size=testset_size, random_state=0)

In [61]:
train_classes

array([3, 8, 9, 2, 9, 7, 8, 5, 2, 9, ..., 7, 9, 6, 7, 7, 0, 4, 8, 1, 8])

In [62]:
test_classes

array([2, 0, 3, 1, 8, 2, 9, 3, 6, 7, ..., 1, 5, 3, 6, 2, 5, 6, 9, 5, 8])

In [69]:
from collections import Counter
cnt = Counter(train_classes)

print "Number of files in each category in TRAIN set:"
for k in sorted(cnt.keys()):
    print k, ":", cnt[k]

Number of files in each category in TRAIN set:
0 : 80
1 : 74
2 : 70
3 : 70
4 : 84
5 : 72
6 : 74
7 : 74
8 : 73
9 : 79


In a Random Split, the number of files per class may be uneven or unbalanced.

The better way to do it is to use a <b>Stratified Split</b>:

In [111]:
# better: Stratified Split retains the class balance in both sets
# from sklearn.model_selection import StratifiedShuffleSplit

if sklearn_version.startswith('0.17'):
    splits = StratifiedShuffleSplit(classes_num, n_iter=1, test_size=testset_size, random_state=0)
else: # >= 0.18:
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=testset_size, random_state=0)
    splits = splitter.split(data, classes_num)

for train_index, test_index in splits:
    print "TRAIN INDEX:", train_index
    print "TEST INDEX:", test_index
    
    # split the data
    train_set = data[train_index]
    test_set = data[test_index]
    
    # and the numeric classes (groundtruth)
    train_classes = classes_num[train_index]
    train_classes_1hot = classes_num_1hot[train_index]  # 1 hot we need for traning
    test_classes = classes_num[test_index]
# Note: this for loop is only executed once, if n_splits==1

print train_set.shape
print test_set.shape
# Note: we will reshape the data later back to matrix form 

TRAIN INDEX: [349 816 960 261 665 517 653 438 170 472 ..., 336 602 117  93 322 434 624  83 118  97]
TEST INDEX: [510 781 478 246 888 793 772 445 199 534 ..., 967 490 486 683 984 390 858 372 982 499]
(750, 3200)
(250, 3200)


In [112]:
cnt = Counter(train_classes)
print "Number of files in each category in TRAIN set:"
for k in sorted(cnt.keys()):
    print k, ":", cnt[k]

Number of files in each category in TRAIN set:
0 : 75
1 : 75
2 : 75
3 : 75
4 : 75
5 : 75
6 : 75
7 : 75
8 : 75
9 : 75


Now the number of files in each category in the training set is equal.

(It is equal because our full set had 100 files in each category; a Stratified Split preserves the relative distribution of instances per category, also if the dataset is unbalanced between the classess.)

# Convolutional Neural Networks

A Convolutional Neural Network (ConvNet or CNN) is a type of (deep) Neural Network that is well-suited for 2D axes data, such as images or spectrograms, as it is optimized for learning from spatial proximity. Its core elements are 2D filter kernels which essentially learn the weights of the Neural Network, and downscaling functions such as Max Pooling.

A CNN can have one or more Convolution layers, each of them having an arbitrary number of N filters (which define the depth of the CNN layer), following typically by a pooling step, which aggregates neighboring pixels together and thus reduces the image resolution by retaining only the maximum values of neighboring pixels.

## Preparing the Data

### Adding the channel

As CNNs were invented for image data (often having 3 color channels), we need to add a dimension for the color channel to the data. 

<b>Spectrograms, are considered like greyscale images, which only have 1 color channel. Still we add the extra dimension, defining just 1 channel.</b>

In Theano, traditionally the color channel is the <b>first</b> dimension in the image shape. 
In Tensorflow, the color channel is the <b>last</b> dimension in the image shape. 

This can be configured now in ~/.keras/keras.json: "image_dim_ordering": "th" or "tf" with "tf" (Tensorflow) being the default image ordering even though you use Theano. Depending on this, use one of the code lines below.

We created an 'if' statement here to check which dimension ordering to use:

In [114]:
n_channels = 1 # 1 for grey-scale, 3 for RGB (in this case usually already present in the data)

if keras.backend.image_dim_ordering() == 'th':
    # Theano ordering (~/.keras/keras.json: "image_dim_ordering": "th")
    train_set = train_set.reshape(train_set.shape[0], n_channels, ydim, xdim)
    test_set = test_set.reshape(test_set.shape[0], n_channels, ydim, xdim)
else:
    # Tensorflow ordering (~/.keras/keras.json: "image_dim_ordering": "tf")
    train_set = train_set.reshape(train_set.shape[0], ydim, xdim, n_channels)
    test_set = test_set.reshape(test_set.shape[0], ydim, xdim, n_channels)

In [115]:
keras.backend.image_dim_ordering()

'tf'

In [116]:
train_set.shape

(750, 40, 80, 1)

In [117]:
test_set.shape

(250, 40, 80, 1)

In [118]:
# we store the new shape of the images in the 'input_shape' variable.
# take all dimensions except the 0th one (which is the number of images)
input_shape = train_set.shape[1:]  
input_shape

(40, 80, 1)

# Creating Neural Network Models in Keras

## Sequential Models

In Keras, one can choose between a Sequential model and a Graph model. Sequential models are the standard case. Graph models are for parallel networks.

## Creating a Single Layer and a Two Layer CNN

Try: (comment/uncomment code in the following code block)
* 1 Layer
* 2 Layer
* more conv_filters
* Dropout

In [239]:
np.random.seed(0) # make results repeatable

model = Sequential()

#conv_filters = 16   # number of convolution filters (= CNN depth)
conv_filters = 32   # number of convolution filters (= CNN depth)

# Layer 1
model.add(Convolution2D(conv_filters, 3, 3, input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2))) 
#model.add(Dropout(0.25)) 

# Layer 2
model.add(Convolution2D(conv_filters, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2))) 

# After Convolution, we have a 16*x*y matrix output
# In order to feed this to a Full(Dense) layer, we need to flatten all data
# Note: Keras does automatic shape inference, i.e. it knows how many (flat) input units the next layer will need,
# so no parameter is needed for the Flatten() layer.
model.add(Flatten()) 

# Full layer
model.add(Dense(256, activation='sigmoid')) 

# Output layer
# For binary/2-class problems use ONE sigmoid unit, for multi-class/multi-label problems use n output units 
# activation should be 'softmax' for multi-class / single-label output, 'sigmoid' for binary or multi-label tasks
model.add(Dense(n_classes,activation='softmax'))

If you get OverflowError: Range exceeds valid bounds in the above box, check the correct Theano vs. Tensorflow ordering in the box before and your keras.json configuration file.

In [156]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 38, 78, 32)    320         convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
maxpooling2d_8 (MaxPooling2D)    (None, 19, 39, 32)    0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 17, 37, 32)    9248        maxpooling2d_8[0][0]             
____________________________________________________________________________________________________
maxpooling2d_9 (MaxPooling2D)    (None, 8, 18, 32)     0           convolution2d_9[0][0]            
___________________________________________________________________________________________

## Training the CNN

In [157]:
# Define a loss function 
loss = 'categorical_crossentropy' 

# Note: for binary classification (2 classes) OR for multi-class problems use:
#loss = 'binary_crossentropy' 

# Optimizer = Stochastic Gradient Descent
optimizer = 'sgd' 

# Compiling the model
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [164]:
# TRAINING the model

# for how many epochs (iterations) to train
epochs = 15

# for training we need the "1 hot encoded" numeric classes of the ground truth
history = model.fit(train_set, train_classes_1hot, batch_size=32, nb_epoch=epochs)

# we keep the history of accuracies on training set

Epoch 1/15
750/750 [==============================] - 5s - loss: 1.6616 - acc: 0.4493     
Epoch 2/15
750/750 [==============================] - 5s - loss: 1.6386 - acc: 0.4387     
Epoch 3/15
750/750 [==============================] - 5s - loss: 1.6132 - acc: 0.4533     
Epoch 4/15
750/750 [==============================] - 5s - loss: 1.5851 - acc: 0.4707     
Epoch 5/15
750/750 [==============================] - 5s - loss: 1.5588 - acc: 0.4867     
Epoch 6/15
750/750 [==============================] - 5s - loss: 1.5385 - acc: 0.4880     
Epoch 7/15
750/750 [==============================] - 5s - loss: 1.5066 - acc: 0.5107     
Epoch 8/15
750/750 [==============================] - 5s - loss: 1.4819 - acc: 0.5187     
Epoch 9/15
750/750 [==============================] - 5s - loss: 1.4579 - acc: 0.5280     
Epoch 10/15
750/750 [==============================] - 5s - loss: 1.4294 - acc: 0.5413     
Epoch 11/15
750/750 [==============================] - 5s - loss: 1.4039 - acc: 0.5467   

In [ ]:
# TODO plot history

#### Accuracy goes up pretty quickly for 1 layer on Train set! Also on Test set?

### Verifying Accuracy on Test Set

In [165]:
test_pred = model.predict_classes(test_set)

250/250 [==============================] - 0s     


In [166]:
# predictions
test_pred

array([1, 1, 2, 4, 2, 1, 7, 4, 4, 1, ..., 8, 5, 1, 6, 2, 6, 2, 6, 5, 7])

In [162]:
# groundtruth
test_classes

array([1, 7, 5, 3, 0, 7, 7, 5, 4, 1, ..., 2, 5, 5, 8, 2, 6, 0, 6, 2, 5])

In [167]:
# 1 layer
accuracy_score(test_classes, test_pred)

0.34799999999999998

In [153]:
# 2 layer
accuracy_score(test_classes, test_pred)

0.36799999999999999

In [67]:
# 2 layer + 32 convolution filters
accuracy_score(test_classes, test_pred)

0.71875

In [85]:
# 2 layer + 32 convolution filters + ReLU + Dropout
accuracy_score(test_classes, test_pred)

0.71875

## Custom Model

In [258]:
model = Sequential()

conv_filters = 16   # number of convolution filters (= CNN depth)

cnn_act = 'sigmoid'

# Layer 1
#model.add(Convolution2D(conv_filters, 3, 3, border_mode='valid', input_shape=input_shape))
model.add(Convolution2D(conv_filters, 4, 12, activation=cnn_act, border_mode='valid', input_shape=input_shape))
#model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(MaxPooling2D(pool_size=(1, 20))) 

# Layer 2
#model.add(Convolution2D(conv_filters, 3, 3, border_mode='valid', input_shape=input_shape))
#model.add(BatchNormalization())
#model.add(Activation('relu')) 
#model.add(MaxPooling2D(pool_size=(2, 2))) 

# After Convolution, we have a 16*x*y matrix output
# In order to feed this to a Full(Dense) layer, we need to flatten all data
# Note: Keras does automatic shape inference, i.e. it knows how many (flat) input units the next layer will need,
# so no parameter is needed for the Flatten() layer.
model.add(Flatten()) 

# Full layer
model.add(Dense(200, activation='sigmoid'))  
model.add(Dropout(0.25))

# Output layer
# For binary/2-class problems use ONE sigmoid unit, for multi-class/multi-label problems use n output units 
# activation should be 'softmax' for multi-class / single-label output, 'sigmoid' for binary or multi-label tasks
model.add(Dense(n_classes,activation='softmax'))

### Training

In [267]:
# Compiling the model

learn_rate = 0.05
optimizer = SGD(lr=learn_rate) #, momentum=momentum) 

model.compile(loss='categorical_crossentropy' , optimizer=optimizer, metrics=['accuracy'])

In [274]:
# TRAINING the model - (you may repeat execution of this cell to FURTHER train the model!)
epochs = 30
history = model.fit(train_set, train_classes_1hot, batch_size=32, nb_epoch=epochs)

Epoch 1/30
750/750 [==============================] - 2s - loss: 1.1851 - acc: 0.5840     
Epoch 2/30
750/750 [==============================] - 2s - loss: 1.1653 - acc: 0.5813     
Epoch 3/30
750/750 [==============================] - 2s - loss: 1.1848 - acc: 0.5627     
Epoch 4/30
750/750 [==============================] - 1s - loss: 1.1689 - acc: 0.5853     
Epoch 5/30
750/750 [==============================] - 1s - loss: 1.1437 - acc: 0.6027     
Epoch 6/30
750/750 [==============================] - 2s - loss: 1.1328 - acc: 0.5987     
Epoch 7/30
750/750 [==============================] - 1s - loss: 1.1565 - acc: 0.5987     
Epoch 8/30
750/750 [==============================] - 1s - loss: 1.1089 - acc: 0.6240     
Epoch 9/30
750/750 [==============================] - 1s - loss: 1.1030 - acc: 0.6147     
Epoch 10/30
750/750 [==============================] - 1s - loss: 1.0911 - acc: 0.6067     
Epoch 11/30
750/750 [==============================] - 1s - loss: 1.1006 - acc: 0.6173   

### Testing

In [275]:
test_pred = model.predict_classes(test_set)

250/250 [==============================] - 0s     


In [276]:
accuracy_score(test_classes, test_pred)

0.496

## Additional Parameters & Techniques

Try: (comment/uncomment code blocks below)
* Adding ReLU activation
* Adding Batch normalization
* Adding Dropout

In [257]:
model = Sequential()

conv_filters = 16   # number of convolution filters (= CNN depth)

# Layer 1
model.add(Convolution2D(conv_filters, 3, 3, border_mode='valid', input_shape=input_shape))
#model.add(Convolution2D(conv_filters, 4, 12, border_mode='valid', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
#model.add(MaxPooling2D(pool_size=(1, 20))) 

# Layer 2
model.add(Convolution2D(conv_filters, 3, 3, border_mode='valid', input_shape=input_shape))
#model.add(BatchNormalization())
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

# After Convolution, we have a 16*x*y matrix output
# In order to feed this to a Full(Dense) layer, we need to flatten all data
# Note: Keras does automatic shape inference, i.e. it knows how many (flat) input units the next layer will need,
# so no parameter is needed for the Flatten() layer.
model.add(Flatten()) 

# Full layer
model.add(Dense(256))  
#model.add(Activation('relu'))
#model.add(Dropout(0.1))

# Output layer
# For binary/2-class problems use ONE sigmoid unit, 
# for multi-class/multi-label problems use n output units and activation='softmax!'
model.add(Dense(n_classes,activation='softmax'))

ValueError: The shape of the input to "Flatten" is not fully defined (got (17, 0, 16). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

### Training

In [250]:
# Compiling the model

optimizer = 'sgd'

#learn_rate = 0.02
#optimizer = SGD(lr=learn_rate) #, momentum=momentum) 

model.compile(loss='categorical_crossentropy' , optimizer=optimizer, metrics=['accuracy'])

In [254]:
# TRAINING the model - (you may repeat execution of this cell to FURTHER train the model!)
epochs = 30
history = model.fit(train_set, train_classes_1hot, batch_size=32, nb_epoch=epochs)

Epoch 1/30
750/750 [==============================] - 2s - loss: 2.1835 - acc: 0.1907     
Epoch 2/30
750/750 [==============================] - 1s - loss: 2.1903 - acc: 0.1987     
Epoch 3/30
750/750 [==============================] - 1s - loss: 2.1851 - acc: 0.1907     
Epoch 4/30
750/750 [==============================] - 1s - loss: 2.1793 - acc: 0.1720     
Epoch 5/30
750/750 [==============================] - 1s - loss: 2.1544 - acc: 0.1947     
Epoch 6/30
750/750 [==============================] - 1s - loss: 2.1471 - acc: 0.2147     
Epoch 7/30
750/750 [==============================] - 2s - loss: 2.1230 - acc: 0.2160     
Epoch 8/30
750/750 [==============================] - 2s - loss: 2.1316 - acc: 0.2107     
Epoch 9/30
750/750 [==============================] - 1s - loss: 2.0883 - acc: 0.2333     
Epoch 10/30
750/750 [==============================] - 1s - loss: 2.1306 - acc: 0.2093     
Epoch 11/30
750/750 [==============================] - 2s - loss: 2.0974 - acc: 0.2320   

### Testing

In [255]:
test_pred = model.predict_classes(test_set)

250/250 [==============================] - 0s     


In [256]:
accuracy_score(test_classes, test_pred)

0.32400000000000001

## Parallel CNNs

The motivation to use parallel CNNs is to use one CNN that detects patterns in the <b>frequencies</b> and another one that captures patterns in the <b>time domain, i.e. rhythm</b> in a combined way to learn better the structures in the sound.

To create parallel CNNs we need a "graph-based" model. In Keras 1.x this is realized via the functional API of the Model() class.
We use it to create two CNN layers that run in parallel to each other and are merged subsequently.

In [235]:
# Input only specifies the input shape
input = Input(input_shape)

# CNN layers
# specify desired number of filters
n_filters = 32 #16 

# The functional API allows to specify the predecessor in (brackets) after the new Layer function call
conv_layer1 = Convolution2D(n_filters, 10, 2, activation='relu')(input)  # a vertical filter
conv_layer2 = Convolution2D(n_filters, 2, 10, activation='relu')(input)  # a horizontal filter


# Pooling layers - equal sized
#maxpool1 = MaxPooling2D(pool_size=(2,2))(conv_layer1)
#maxpool2 = MaxPooling2D(pool_size=(2,2))(conv_layer2)

# ALTERNATIVE: Pooling layers - complementary to vertical/horizontal filter
maxpool1 = MaxPooling2D(pool_size=(1,2))(conv_layer1)
maxpool2 = MaxPooling2D(pool_size=(2,1))(conv_layer2)


# we have to flatten the Pooling output in order to be concatenated
poolflat1 = Flatten()(maxpool1)
poolflat2 = Flatten()(maxpool2)

# Merge the 2
merged = merge([poolflat1, poolflat2], mode='concat')

full = Dense(256, activation='sigmoid')(merged)
output_layer = Dense(n_classes, activation='softmax')(full)

# finally create the model
model = Model(input=input, output=output_layer)

In [236]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 40, 80, 1)     0                                            
____________________________________________________________________________________________________
convolution2d_28 (Convolution2D) (None, 31, 79, 32)    672         input_6[0][0]                    
____________________________________________________________________________________________________
convolution2d_29 (Convolution2D) (None, 39, 71, 32)    672         input_6[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_28 (MaxPooling2D)   (None, 31, 39, 32)    0           convolution2d_28[0][0]           
___________________________________________________________________________________________

### Training

In [237]:
# Compiling the model

learn_rate = 0.02
optimizer = SGD(lr=learn_rate) #, momentum=momentum) 

model.compile(loss='categorical_crossentropy' , optimizer=optimizer, metrics=['accuracy'])

In [238]:
# TRAINING the model
epochs = 15
history = model.fit(train_set, train_classes_1hot, batch_size=32, nb_epoch=epochs)

Epoch 1/15
750/750 [==============================] - 11s - loss: 2.1489 - acc: 0.2173    
Epoch 2/15
750/750 [==============================] - 11s - loss: 1.9068 - acc: 0.3107    
Epoch 3/15
750/750 [==============================] - 10s - loss: 1.7642 - acc: 0.3960    
Epoch 4/15
750/750 [==============================] - 11s - loss: 1.6289 - acc: 0.4667    
Epoch 5/15
750/750 [==============================] - 12s - loss: 1.5371 - acc: 0.4840    
Epoch 6/15
750/750 [==============================] - 13s - loss: 1.4501 - acc: 0.4920    
Epoch 7/15
750/750 [==============================] - 13s - loss: 1.3461 - acc: 0.5547    
Epoch 8/15
750/750 [==============================] - 12s - loss: 1.2918 - acc: 0.5693    
Epoch 9/15
750/750 [==============================] - 10s - loss: 1.1987 - acc: 0.6133    
Epoch 10/15
750/750 [==============================] - 10s - loss: 1.1592 - acc: 0.6200    
Epoch 11/15
750/750 [==============================] - 10s - loss: 1.1185 - acc: 0.6280  

### Testing

In [222]:
#test_pred = model.predict_classes(test_set)

# THE GRAPH MODEL DOES NOT SUPPORT .predict_classes
# we use model.predict

test_pred = model.predict(test_set)

In [223]:
# it predicts the probabilities per class:
test_pred

array([[ 0.00000088,  0.99960399,  0.0000328 ,  0.00000023,  0.        ,  0.00035961,  0.        ,  0.00000055,  0.00000029,  0.00000165],
       [ 0.1437121 ,  0.03942114,  0.08495273,  0.05749788,  0.00098327,  0.43678114,  0.00377144,  0.05442544,  0.1472127 ,  0.03124217],
       [ 0.1809573 ,  0.01344254,  0.19387296,  0.04931471,  0.007558  ,  0.16700505,  0.00760951,  0.00200711,  0.06899887,  0.30923396],
       [ 0.00307811,  0.00009272,  0.0044343 ,  0.13793397,  0.51605999,  0.18190596,  0.02145181,  0.02988708,  0.06580672,  0.03934935],
       [ 0.20178558,  0.21113855,  0.08383352,  0.18441004,  0.00160872,  0.02205589,  0.01210712,  0.03152585,  0.02146892,  0.23006581],
       [ 0.20334674,  0.03489076,  0.01057608,  0.02076097,  0.0004238 ,  0.06517673,  0.02537389,  0.00362619,  0.48641843,  0.14940642],
       [ 0.00178482,  0.00014169,  0.12583028,  0.03404034,  0.00496965,  0.01788303,  0.00012243,  0.80130893,  0.01154513,  0.00237368],
       [ 0.00540129,  0.000

In [224]:
# we get the predicted class from the maximum value in the probabilities, using argmax to get the class number
test_pred = np.argmax(test_pred, axis=1)
test_pred

array([1, 5, 9, 4, 9, 8, 7, 2, 3, 1, ..., 8, 5, 5, 6, 8, 6, 2, 6, 2, 7])

In [225]:
accuracy_score(test_classes, test_pred)

0.47999999999999998

## Further Practice

* add more layers
* vary:
 * number of filters
 * filter sizes (shapes)
 * max pooling
 * activation functions: sigmoid, tanh, relu, prelu, elu, ...
* try other optimizers: 'from keras.optimizers import SGD, RMSprop, Adagrad'